In [1]:
# Modules
import csv
import os
import pandas as pd

inpath = os.path.join('.','Input_Data')
outpath = os.path.join('..','Load')

In [2]:
# Store filepath in a variable
sales_file = os.path.join(inpath,"sales.xls")
inventory_file = os.path.join(inpath,"inventory_sales.xls")

In [3]:
def pull_year(year):
    #Read Data from excel files for the table worksheets in years
    xls=pd.ExcelFile(sales_file)
    sales=pd.read_excel(sales_file, f"{year}")
    
    #Put Data into a DataFrames
    sales_df=pd.DataFrame(sales)
    
    #Define counters for not_adjusted and adjusted rows
    not_adjusted_row = -1
    adjusted_row = -1
    
    #for loop to count row numbers for not_adjusted and adjusted rows
    for row in range(len(sales)):
        if sales_df.loc[row,'Unnamed: 1'] == 'NOT ADJUSTED':
            not_adjusted_row = row
        if sales_df.loc[row,'Unnamed: 1'] in ('ADJUSTED(2)', 'ADJUSTED(1)'):
            adjusted_row = row
    
    #locate the rows between not-adjusted and adjusted
    clean_sales = sales_df.iloc[not_adjusted_row+1:adjusted_row-1, 0:14]
    
    clean_sales.columns=["NAICS_Code", "Business","Jan", "Feb", "Mar", "Apr", "May", \
                         "Jun", "Jul", "Aug", "Sep", "Oct", "Nov","Dec"] 
    
    clean_sales["Year"] = f"{year}"

    clean_sales = clean_sales[["NAICS_Code", "Year","Jan", "Feb", "Mar", "Apr", "May", \
                               "Jun", "Jul", "Aug", "Sep", "Oct", "Nov","Dec"] ]
    
    return clean_sales

sales_2018 = pull_year(2018)
sales_2017 = pull_year(2017)
sales_2016 = pull_year(2016)
sales_2015 = pull_year(2015)
sales_2014 = pull_year(2014)



In [4]:
## Combining all tables to create one sales data frame ##
sales=pd.concat([sales_2018, sales_2017, sales_2016, sales_2015, sales_2014], axis=0, ignore_index=True)
print(f"Sales dataframe has {len(sales)} rows.")
sales.head()

Sales dataframe has 320 rows.


,NAICS_Code,Year,Jan,Feb,Mar,Apr,May,Jun,Jul,Aug,Sep,Oct,Nov,Dec
0,NaN,2018,445484,437005,510380,482412,530082,510029,508010,523933,481094,506360,522804,563497
1,NaN,2018,357749,347705,401041,383280,421168,405277,404116,415396,384778,406873,427005,461535
2,NaN,2018,407926,401292,469422,440936,483790,464191,461683,476746,437161,459751,481131,525663
3,NaN,2018,320191,311992,360083,341804,374876,359439,357789,368209,340845,360264,385332,423701
4,NaN,2018,390689,381932,446512,421678,465377,445793,443977,458781,420602,444271,463403,500260


In [5]:
## Using the melt function to massage data set into a vertical format sorted by NAICS_Code and Year ##
sales_data = sales.melt(id_vars=["NAICS_Code","Year"],var_name=["Month"],value_name="Total_Sales")
sales_data.sort_values(by=['Year','Month'])
sales_data.head()

,NAICS_Code,Year,Month,Total_Sales
0,NaN,2018,Jan,445484
1,NaN,2018,Jan,357749
2,NaN,2018,Jan,407926
3,NaN,2018,Jan,320191
4,NaN,2018,Jan,390689


In [6]:
### Cleaning Data: Eliminating total columns and rows with no sales information ##
# Delete rows wihtout NAICS codes. These were total rows.
sales_data = sales_data.dropna()

# Delete row where the sales were suppressed. These do not have usable information.
sales_data = sales_data[sales_data["Total_Sales"] != "(S)"]

# Look at the final data
sales_data.head()

,NAICS_Code,Year,Month,Total_Sales
7,441,2018,Jan,87735
8,"4411,4412",2018,Jan,80723
9,4411,2018,Jan,76671
10,44111,2018,Jan,67953
11,44112,2018,Jan,8718


In [7]:
# Export file as a CSV, without the Pandas index, but with the header ##
outfile = os.path.join(outpath,'sales_data.csv')
sales_data.to_csv(outfile,index=False)